In [1]:
import requests
import pyarabic.araby as araby # for processing arabic 
import re

In [2]:
lines = []
with open('actor_dict_ar_v2.txt') as file:
    for i in file : 
        lines.append(i)
    
lines[0:10]

['غزة  [PSE]\n',
 'الجيش  [MIL]\n',
 'أبو بلال\n',
 '\t[PSECVLELI 19840502-18000101]\n',
 'جلال حسنين\n',
 '\t[LBNCVLREF 19840502-18000101]\n',
 'وزارة الدفاع الأميركية  [USAGOV]\n',
 'شيخة لبنى القاسمي\n',
 '\t[AREGOV 20001106-18000101]\n',
 'الاتحاد الأفريقي  [IGOGOV]\n']

In [3]:
#udpipe stuff : i love reproducable code  !  
def udpipe(string) :
    # Prepping String 
    words  = re.findall(u'[\u0600-\u06FF]+', string) #getting arabic characters 
    data = ' '.join(words)
    
    pipe_base_url = 'http://lindat.mff.cuni.cz/services/udpipe/api/process?tokenizer&tagger' 
    attributes = {} 
    attributes['model'] = 'arabic-ud-2.0-170801'
    attributes['data'] = data

    data = requests.get(pipe_base_url , attributes)
    result = data.json()['result'].split('\n')
    udpipe_results = [re.findall(u'[\u0600-\u06FF]+', i) for i in result] # cleaning 
    udpipe_results = [i for i in udpipe_results if i !=[]]
    
    return udpipe_results

def udpipe_reconstruct(original_text) : 
    udpipe_results = udpipe(original_text)
    for i in range(1, len(udpipe_results)):
        if len(udpipe_results[i]) == 1 : # composite verbs get the next two words
            original_text =original_text.replace(udpipe_results[i][0] , '{} {}'.format(udpipe_results[i+1][1],udpipe_results[i+2][1]))
        else :
            original_text =original_text.replace(udpipe_results[i][0] , udpipe_results[i][1])
    original_text = araby.strip_tashkeel(original_text)
    return original_text


def master_reconstruct_input(text_input , input_type): 
    # type 0: none-rule just do just do udpipe_reconstruction
    # type 1 : fix the rule + do udpipe_reconstruction - no need for this here 
    if input_type == 0:
        return udpipe_reconstruct(text_input)

    

In [7]:
lines2 = []
for i in lines:
    lines2.append(master_reconstruct_input(i,0))
print('#Done!!')

#Done!!


In [8]:
with open('actor_dic_ar_v3.txt', 'w') as output : 
    for i in lines2 : 
        output.write(i)
    

## fix formats (Adding underscore instead of spaces) 

In [27]:
import re
actors = [] 
with open('actor_dic_ar_v3.txt') as finput : 
        for i in finput  : 
            actors.append(i)
actors[:10]

['غزة  [PSE]\n',
 'جيش  [MIL]\n',
 'ابو بلال\n',
 '\t[PSECVLELI 19840502-18000101]\n',
 'جلال حسنين\n',
 '\t[LBNCVLREF 19840502-18000101]\n',
 'وزارة دفاع أميركي  [USAGOV]\n',
 'شيخة لبنى القاسمي\n',
 '\t[AREGOV 20001106-18000101]\n',
 'اتحاد أفريقي  [IGOGOV]\n']

In [28]:
x = '_'.join(re.findall(r'[\u0600-\u06FF]+', actors[2])) +'_'
x

'ابو_بلال_'

In [29]:
for i in range(len(actors)) : 
    ar_exp = re.findall(r'[\u0600-\u06FF]+', actors[i])
    if not ar_exp or len(ar_exp)==1 :
        continue
    new_ar_exp = '_'.join(ar_exp) + '_'
#     print('ar_exp {}'.format(ar_exp))
#     print('new_ar_exp {}'.format(new_ar_exp))
    
#     print('before {} '.format(i))
    actors[i] = actors[i].replace(' '.join(ar_exp) , new_ar_exp)
#     print('after {} '.format(i))
    
    
    
actors[:10]

['غزة  [PSE]\n',
 'جيش  [MIL]\n',
 'ابو_بلال_\n',
 '\t[PSECVLELI 19840502-18000101]\n',
 'جلال_حسنين_\n',
 '\t[LBNCVLREF 19840502-18000101]\n',
 'وزارة_دفاع_أميركي_  [USAGOV]\n',
 'شيخة_لبنى_القاسمي_\n',
 '\t[AREGOV 20001106-18000101]\n',
 'اتحاد_أفريقي_  [IGOGOV]\n']

In [30]:
with open('actor_dict_ar_v4.txt', 'w') as output : 
    for i in actors:
        output.write(i)